## Task 2: CORE SECTION (ICP Algorithm)

In [1]:
import sys,os

# Path to Resources
RES_PATH = './bunny/data_rough_aligned'
COR_PATH = './bunny/correspondences'

# Checking if the path is found locally
if not os.path.exists(RES_PATH):
    print( 'cannot find mesh resources dir, please update RES_PATH')
    exit(1)
else:
    print('found resources')

# Loading libraries for basic geometry processing
import trimesh
import numpy as np
from sklearn.neighbors import KDTree

found resources


### Registration with multiple scans

Now given multiple scans M1,M2,...M5 align all of them to a common global coordinate frame.

In [5]:
# Load point cloud models M1 ~ M5
mesh_fp1 = os.path.join(RES_PATH,'bun000.ply')
assert os.path.exists(mesh_fp1), 'Cannot find:' + mesh_fp1
M1 = trimesh.load(mesh_fp1)
print('M1 shape of Vertices:', M1.vertices.shape)

mesh_fp2 = os.path.join(RES_PATH,'bun090.ply')
assert os.path.exists(mesh_fp2), 'Cannot find:' + mesh_fp2
M2 = trimesh.load(mesh_fp2)
print('M2 shape of Vertices:', M2.vertices.shape)

mesh_fp3 = os.path.join(RES_PATH,'bun180.ply')
assert os.path.exists(mesh_fp3), 'Cannot find:' + mesh_fp3
M3 = trimesh.load(mesh_fp3)
print('M3 shape of Vertices:', M3.vertices.shape)

mesh_fp4 = os.path.join(RES_PATH,'bun270.ply')
assert os.path.exists(mesh_fp4), 'Cannot find:' + mesh_fp4
M4 = trimesh.load(mesh_fp4)
print('M4 shape of Vertices:', M4.vertices.shape)

mesh_fp5 = os.path.join(RES_PATH,'top2.ply')
assert os.path.exists(mesh_fp5), 'Cannot find:' + mesh_fp5
M5 = trimesh.load(mesh_fp5)
print('M5 shape of Vertices:', M5.vertices.shape)

mesh_fp6 = os.path.join(RES_PATH,'chin.ply')
assert os.path.exists(mesh_fp6), 'Cannot find:' + mesh_fp6
M6 = trimesh.load(mesh_fp6)
print('M6 shape of Vertices:', M6.vertices.shape)

# Create a list to store models to be registered
models = [M1, M2, M3, M4, M5, M6]
print(models)

M1 shape of Vertices: (40256, 3)
M2 shape of Vertices: (40091, 3)
M3 shape of Vertices: (30373, 3)
M4 shape of Vertices: (40247, 3)
M5 shape of Vertices: (31697, 3)
[<trimesh.PointCloud(vertices.shape=(40256, 3), name=`bun000.ply`)>, <trimesh.Trimesh(vertices.shape=(40091, 3), faces.shape=(79057, 3), name=`bun045.ply`)>, <trimesh.Trimesh(vertices.shape=(30373, 3), faces.shape=(59629, 3), name=`bun090.ply`)>, <trimesh.Trimesh(vertices.shape=(40247, 3), faces.shape=(79284, 3), name=`bun180.ply`)>, <trimesh.Trimesh(vertices.shape=(31697, 3), faces.shape=(62335, 3), name=`bun270.ply`)>]


In [6]:
def align_multiple_scans(scans, max_iterations=100, tolerance=1e-6):
    """
    Align multiple scans to a common global coordinate frame.

    Args:
    - scans: a list of numpy arrays representing the vertices of each scan (shape: [N_scan, Nx3])
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: tolerance for convergence

    Returns:
    - aligned_scans: a list of aligned scans (as trimesh.pointcloud.PointCloud objects)
    """

    # Initialize list to store aligned scans
    aligned_scans = []
    aligned_scans.append(scans[0])

    for i, scan in enumerate(scans[1:], start=1):
        print(f'Processing model M{i+1}...')
        # Copy the vertices to avoid modifying the original data
        scan_vertices = scan.vertices.copy()

        # Set reference scan as the previous one
        reference_scan = aligned_scans[i-1].vertices.copy()

        # Initialize rotation matrix R and translation vector t
        R = np.eye(3)
        t = np.zeros((3, 1))

        # Perform ICP to align the current scan to the reference scan
        for iteration in range(1, max_iterations+1):
            # Find the nearest neighbors between the current scan and the reference scan
            tree = KDTree(reference_scan)
            _, indices = tree.query(scan_vertices)

            # Extract corresponding points from the reference scan and the current scan
            P = reference_scan[indices].squeeze()
            Q = scan_vertices

            # Compute the centroid of the matched points
            centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
            centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

            # Compute the cross-covariance matrix
            A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

            # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
            U, _, Vt = np.linalg.svd(A)
            R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @U.T

            # Compute the optimal translation vector t
            t = centroid_P.T - R @ centroid_Q.T

            # Apply transformation to the current scan for the next iteration
            scan_vertices = (R @ scan_vertices.T + t).T

            # Check convergence
            if np.linalg.norm(t) < tolerance:
                print('Converge at iteration', iteration)
                break

        if iteration == max_iterations:
            print(iteration, 'iterations complete.')

        # Store the aligned scan
        aligned_scan = trimesh.points.PointCloud(scan_vertices)
        aligned_scans.append(aligned_scan)

    return aligned_scans

In [7]:
aligned_scans = align_multiple_scans(models)

for i, aligned_scan in enumerate(aligned_scans):
    directory = './results/multiple_scans/method1/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_M{i+1}.ply"
    filepath = os.path.join(directory, filename)
    aligned_scan.export(filepath)

Processing model M2...


Converge at iteration 32
Processing model M3...
Converge at iteration 70
Processing model M4...
Converge at iteration 99
Processing model M5...
Converge at iteration 60


In [9]:
def align_multiple_scans_2(scans, max_iterations=100, tolerance=1e-6):
    """
    Align multiple scans to a common global coordinate frame.

    Args:
    - scans: a list of numpy arrays representing the vertices of each scan (shape: [N_scan, Nx3])
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: tolerance for convergence

    Returns:
    - aligned_scans: a list of aligned scans (as trimesh.pointcloud.PointCloud objects)
    """

    # Initialize list to store aligned scans
    aligned_scans = []
    aligned_scans.append(scans[0])

    for i, scan in enumerate(scans[1:], start=1):
        print(f'Processing model M{i+1}...')
        # Copy the vertices to avoid modifying the original data
        scan_vertices = scan.vertices.copy()

        # Set reference scan as all the other scans
        other_scans = scans[:i] + scans[i+1:]
        reference_scan = np.concatenate([s.vertices.copy() for s in other_scans])

        # Initialize rotation matrix R and translation vector t
        R = np.eye(3)
        t = np.zeros((3, 1))

        # Perform ICP to align the current scan to the reference scan
        for iteration in range(1, max_iterations+1):
            # Find the nearest neighbors between the current scan and the reference scan
            tree = KDTree(reference_scan)
            _, indices = tree.query(scan_vertices)

            # Extract corresponding points from the reference scan and the current scan
            P = reference_scan[indices].squeeze()
            Q = scan_vertices

            # Compute the centroid of the matched points
            centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
            centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

            # Compute the cross-covariance matrix
            A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

            # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
            U, _, Vt = np.linalg.svd(A)
            R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @ U.T

            # Compute the optimal translation vector t
            t = centroid_P.T - R @ centroid_Q.T

            # Apply transformation to the current scan for the next iteration
            scan_vertices = (R @ scan_vertices.T + t).T

            # Check convergence
            if np.linalg.norm(t) < tolerance:
                print('Converge at iteration', iteration)
                break

        if iteration == max_iterations:
            print(iteration, 'iterations complete.')

        # Store the aligned scan
        aligned_scan = trimesh.points.PointCloud(scan_vertices)
        aligned_scans.append(aligned_scan)

    return aligned_scans

In [10]:
aligned_scans = align_multiple_scans_2(models)

for i, aligned_scan in enumerate(aligned_scans):
    directory = './results/multiple_scans/method2/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_M{i+1}.ply"
    filepath = os.path.join(directory, filename)
    aligned_scan.export(filepath)

Processing model M2...
Converge at iteration 64
Processing model M3...
Converge at iteration 55
Processing model M4...
Converge at iteration 54
Processing model M5...
Converge at iteration 39


## Simultaneous Multiview Registration

In [1]:
import sys,os

# Path to Resources
RES_PATH = './bunny/data_rough_aligned'
COR_PATH = './bunny/correspondences'

# Checking if the path is found locally
if not os.path.exists(RES_PATH):
    print( 'cannot find mesh resources dir, please update RES_PATH')
    exit(1)
else:
    print('found resources')

# Loading libraries for basic geometry processing
import trimesh
import numpy as np
from sklearn.neighbors import KDTree

found resources


In [10]:
num_views = 6  # (M) There are 6 views in the example (V1 to V6)
num_correspondences = 12  # (P) There are 12 correspondence sets (S1 to S12)

In [2]:
# Load multi-views point cloud models M1 ~ M6
mesh_fp1 = os.path.join(RES_PATH,'bun000.ply')
assert os.path.exists(mesh_fp1), 'Cannot find:' + mesh_fp1
M1 = trimesh.load(mesh_fp1)
print('M1 shape of Vertices:', M1.vertices.shape)

mesh_fp2 = os.path.join(RES_PATH,'bun090.ply')
assert os.path.exists(mesh_fp2), 'Cannot find:' + mesh_fp2
M2 = trimesh.load(mesh_fp2)
print('M2 shape of Vertices:', M2.vertices.shape)

mesh_fp3 = os.path.join(RES_PATH,'bun180.ply')
assert os.path.exists(mesh_fp3), 'Cannot find:' + mesh_fp3
M3 = trimesh.load(mesh_fp3)
print('M3 shape of Vertices:', M3.vertices.shape)

mesh_fp4 = os.path.join(RES_PATH,'bun270.ply')
assert os.path.exists(mesh_fp4), 'Cannot find:' + mesh_fp4
M4 = trimesh.load(mesh_fp4)
print('M4 shape of Vertices:', M4.vertices.shape)

mesh_fp5 = os.path.join(RES_PATH,'top2.ply')
assert os.path.exists(mesh_fp5), 'Cannot find:' + mesh_fp5
M5 = trimesh.load(mesh_fp5)
print('M5 shape of Vertices:', M5.vertices.shape)

mesh_fp6 = os.path.join(RES_PATH,'chin.ply')
assert os.path.exists(mesh_fp6), 'Cannot find:' + mesh_fp6
M6 = trimesh.load(mesh_fp6)
print('M6 shape of Vertices:', M6.vertices.shape)

# Create a list to store models to be registered
models = [M1.vertices.copy(), M2.vertices.copy(), M3.vertices.copy(), M4.vertices.copy(), M5.vertices.copy(), M6.vertices.copy()]

M1 shape of Vertices: (40256, 3)
M2 shape of Vertices: (30373, 3)
M3 shape of Vertices: (40247, 3)
M4 shape of Vertices: (31697, 3)
M5 shape of Vertices: (38297, 3)
M6 shape of Vertices: (37737, 3)


In [3]:
# Load multi-views point cloud models C1_i ~ C12_i

# Create a list to store correspondences
X = []
Y = []

file_names = [
    'bun000_CPSet000_090.ply',
    'bun090_CPSet000_090.ply',
    'bun090_CPSet090_180.ply',
    'bun180_CPSet090_180.ply',
    'bun180_CPSet180_270.ply',
    'bun270_CPSet180_270.ply',
    'bun270_CPSet000_270.ply',
    'bun000_CPSet000_270.ply',
    'bun000_CPSet000_top.ply',
    'top_CPSet000_top.ply',
    'bun090_CPSet090_top.ply',
    'top_CPSet090_top.ply',
    'bun180_CPSet180_top.ply',
    'top_CPSet180_top.ply',
    'bun270_CPSet270_top.ply',
    'top_CPSet270_top.ply',
    'bun000_CPSet000_chin.ply',
    'chin_CPSet000_chin.ply',
    'bun090_CPSet090_chin.ply',
    'chin_CPSet090_chin.ply',
    'bun180_CPSet180_chin.ply',
    'chin_CPSet180_chin.ply',
    'bun270_CPSet270_chin.ply',
    'chin_CPSet270_chin.ply'
]

for i, filename in enumerate(file_names):
    # 构造PLY文件的完整路径
    cor_fp = os.path.join(COR_PATH, filename)
    
    # 检查文件是否存在
    assert os.path.exists(cor_fp), 'Cannot find:' + cor_fp
    
    # 使用trimesh库加载PLY文件
    cor_mesh = trimesh.load(cor_fp)

    # 打印顶点数组的形状信息
    print(filename, 'shape of Vertices:', cor_mesh.vertices.shape)
    
    # 根据文件顺序将顶点数组分别存储在X和Y列表中
    if i % 2 == 0:  # 当X和Y长度相等时，将顶点数组存储在X列表中
        X.append(cor_mesh.vertices.copy())
    else:  # 当X和Y长度不相等时，将顶点数组存储在Y列表中
        Y.append(cor_mesh.vertices.copy())

# 打印X和Y列表中的顶点数组数量
print("Total number of vertex arrays in X:", len(X))
print("Total number of vertex arrays in Y:", len(Y))


bun000_CPSet000_090.ply shape of Vertices: (30379, 3)
bun090_CPSet000_090.ply shape of Vertices: (40256, 3)
bun090_CPSet090_180.ply shape of Vertices: (40251, 3)
bun180_CPSet090_180.ply shape of Vertices: (30379, 3)
bun180_CPSet180_270.ply shape of Vertices: (31701, 3)
bun270_CPSet180_270.ply shape of Vertices: (40251, 3)
bun270_CPSet000_270.ply shape of Vertices: (40256, 3)
bun000_CPSet000_270.ply shape of Vertices: (31701, 3)
bun000_CPSet000_top.ply shape of Vertices: (38298, 3)
top_CPSet000_top.ply shape of Vertices: (40256, 3)
bun090_CPSet090_top.ply shape of Vertices: (38298, 3)
top_CPSet090_top.ply shape of Vertices: (30379, 3)
bun180_CPSet180_top.ply shape of Vertices: (38298, 3)
top_CPSet180_top.ply shape of Vertices: (40251, 3)
bun270_CPSet270_top.ply shape of Vertices: (38298, 3)
top_CPSet270_top.ply shape of Vertices: (31701, 3)
bun000_CPSet000_chin.ply shape of Vertices: (37738, 3)
chin_CPSet000_chin.ply shape of Vertices: (40256, 3)
bun090_CPSet090_chin.ply shape of Vertic

In [20]:
# Find the nearest neighbors between the corresponding scans
for mu in range(num_correspondences):
    tree = KDTree(Y[mu])
    _, indices = tree.query(X[mu])

    # Extract nearest points as the correspondences
    Y[mu] = Y[mu][indices].squeeze()

In [22]:
# Weights all initialized to 1 for now
weights = [np.ones(len(X[mu])) for mu in range(num_correspondences)]

### Define the View–Correspondence Mapping

In [23]:
# Given the View–Correspondence Mapping Table, construct the Cα and Cβ matrices.
# These are "view selection matrices" that map correspondence sets to their respective views.

# Define the mappings as provided in the table
alpha_mapping = [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
beta_mapping = [2, 3, 4, 1, 5, 5, 5, 5, 6, 6, 6, 6]

# Initialize the selection matrices with zeros
C_alpha = np.zeros((3 * num_views, 3 * num_correspondences))
C_beta = np.zeros((3 * num_views, 3 * num_correspondences))

# Fill in the selection matrices based on the mappings
for mu in range(num_correspondences):
    view_alpha = alpha_mapping[mu] - 1  # Adjust for zero-based indexing
    view_beta = beta_mapping[mu] - 1  # Adjust for zero-based indexing

    # Each correspondence set affects 3 rows and 3 columns, for x, y, z components
    C_alpha[3*view_alpha:3*(view_alpha+1), 3*mu:3*(mu+1)] = np.eye(3)
    C_beta[3*view_beta:3*(view_beta+1), 3*mu:3*(mu+1)] = np.eye(3)

### Construct Q Matrix

In [31]:
def compute_Q(X, Y, weights):
    ## Compute Q_R
    # Initialize the H matrices as zero matrices of the appropriate size
    Hxx = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Hyy = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Hxy = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Hyx = np.zeros((3 * num_correspondences, 3 * num_correspondences))

    # Calculate the H matrices
    for mu in range(num_correspondences):
        # Weight each point's contribution to the outer product
        weighted_X = X[mu] * weights[mu][:, np.newaxis]
        weighted_Y = Y[mu] * weights[mu][:, np.newaxis]

        Hxx_mu = weighted_X.T @ X[mu]
        Hyy_mu = weighted_Y.T @ Y[mu]
        Hxy_mu = weighted_X.T @ Y[mu]
        Hyx_mu = Hxy_mu.T  # Hyx is the transpose of Hxy

        # Place the computed matrices in their appropriate blocks in the H matrices
        Hxx[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = Hxx_mu
        Hyy[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = Hyy_mu
        Hxy[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = Hxy_mu
        Hyx[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = Hyx_mu

    # Using the computed matrices to calculate Q_R
    Q_R = C_alpha @ Hxx @ C_alpha.T + C_beta @ Hyy @ C_beta.T - C_alpha @ Hxy @ C_beta.T - C_beta @ Hyx @ C_alpha.T

    ## Compute Q_R_T
    # Step 1: Compute W_mu as the sum of weights for each correspondence
    W_mu = np.array([np.sum(weights[mu]) for mu in range(num_correspondences)])

    # Step 2: Create the W matrix
    W = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    for mu in range(num_correspondences):
        W[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = np.eye(3) * W_mu[mu]

    # Step 3: Compute the matrix Gamma
    N = np.eye(C_alpha.shape[1])  # ? unkown definition, set to identity for now
    Gamma_matrix = W @ (C_alpha - C_beta).T @ np.linalg.pinv((C_alpha - C_beta) @ N @ (C_alpha - C_beta).T) @ (C_alpha - C_beta) @ W
    # Extract gamma_coefficients from Gamma's diagonal blocks
    gamma_coefficients = np.array([Gamma_matrix[3*mu, 3*mu] for mu in range(num_correspondences)])

    # Step 4: Compute the weighted centroids for each correspondence set
    weighted_centroids_x = [np.average(X[mu], weights=weights[mu], axis=0) for mu in range(num_correspondences)]
    weighted_centroids_y = [np.average(Y[mu], weights=weights[mu], axis=0) for mu in range(num_correspondences)]

    # Step 5: Compute the G matrices
    Gxx = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Gyy = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Gxy = np.zeros((3 * num_correspondences, 3 * num_correspondences))
    Gyx = np.zeros((3 * num_correspondences, 3 * num_correspondences))

    for mu in range(num_correspondences):
        Gxx[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = np.outer(weighted_centroids_x[mu], weighted_centroids_x[mu]) * gamma_coefficients[mu]
        Gyy[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = np.outer(weighted_centroids_y[mu], weighted_centroids_y[mu]) * gamma_coefficients[mu]
        Gxy[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = np.outer(weighted_centroids_x[mu], weighted_centroids_y[mu]) * gamma_coefficients[mu]
        Gyx[3*mu:3*(mu+1), 3*mu:3*(mu+1)] = np.outer(weighted_centroids_y[mu], weighted_centroids_x[mu]) * gamma_coefficients[mu]

    # Now use the G matrices and the view selection matrices to compute Q_R_T
    Q_R_T = -C_alpha @ Gxx @ C_alpha.T + C_alpha @ Gxy @ C_beta.T + C_beta @ Gyx @ C_alpha.T - C_beta @ Gyy @ C_beta.T

    # Compute Q matrix
    Q = Q_R + Q_R_T

    return Q, weighted_centroids_x, weighted_centroids_y, W

### Iterative algorithm to solve for R and T

In [32]:
def compute_R_and_T(R, T, Q, weighted_centroids_x, weighted_centroids_y, W):
    ## Solve for R
    # Iterate over each view to optimize its rotation matrix
    for j in range(num_views):
        # Partition R into R_L, R_j, and R_U
        R_L = R[:, :3*j]
        R_j = R[:, 3*j:3*(j+1)]
        R_U = R[:, 3*(j+1):]

        # Partition Q into corresponding blocks
        Q_jL = Q[3*j:3*(j+1), :3*j]
        Q_jU = Q[3*j:3*(j+1), 3*(j+1):]

        # Construct the S_j matrix for the j-th rotation
        S_j = Q_jL @ R_L.T + Q_jU @ R_U.T

        # Perform SVD of -S_j
        U, _, Vt = np.linalg.svd(-S_j)

        # Calculate the optimal R_j and update the corresponding block in R
        R[:, 3*j:3*(j+1)] = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @ U.T

    ## Solve for T
    # Initialize Z as an empty list to collect the blocks
    Z_blocks = []

    # Construct Z by concatenating the relative displacements of the rotated centroids
    for mu in range(num_correspondences):
        rotated_centroid_x = R @ C_alpha[:, 3*mu:3*(mu+1)] @ weighted_centroids_x[mu]
        rotated_centroid_y = R @ C_beta[:, 3*mu:3*(mu+1)] @ weighted_centroids_y[mu]
        Z_block = rotated_centroid_x - rotated_centroid_y
        Z_blocks.append(Z_block)

    Z = np.concatenate(Z_blocks, axis=0)  # Combine the blocks vertically

    # Construct matrices A and B
    A = (C_alpha - C_beta) @ W @ (C_alpha - C_beta).T
    B = (C_alpha - C_beta) @ W @ Z

    # Solve for T_min using the pseudo-inverse of A
    T_min = -np.linalg.pinv(A) @ B

    T = T_min.reshape((3*num_views, 1))

    return R, T

### Simultaneous registration

In [33]:
def simultaneous_multiscans_registration(scans, X, Y, weights, max_iterations=100, tolerance=1e-6):
    R = np.hstack([np.eye(3)] * num_views)
    T = np.zeros((3*num_views, 1))

    for iteration in range(1, max_iterations+1):
        Q, weighted_centroids_x, weighted_centroids_y, W = compute_Q(X, Y, weights)
        R, T = compute_R_and_T(R, T, Q, weighted_centroids_x, weighted_centroids_y, W)

        # Update each model
        for j in range(num_views):
            # Extract the rotation matrix R_j for view j
            R_j = R[:, 3*j:3*(j+1)]
            
            # Extract the translation vector T_j for view j
            T_j = T[3*j:3*(j+1)]

            scans[j] = (R_j @ models[j].T + T_j).T

        if iteration == max_iterations:
            print(iteration, 'Maximum iteration reached.')

    return scans

In [34]:
models = simultaneous_multiscans_registration(models, X, Y, weights)

for i, model_vertices in enumerate(models):
    directory = './results/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_M{i+1}.ply"
    filepath = os.path.join(directory, filename)
    model = trimesh.points.PointCloud(model_vertices)
    model.export(filepath)

100 Maximum iteration reached.
